![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Agentic tasks with AutoGen and Redis

This notebook demonstrates how to build an agent using Microsoft's [AutoGen](https://microsoft.github.io/autogen/stable//index.html) agent framework and the RedisMemory integration.

We'll define an agent, give it access to tools and memory, then set in on a task to see how it uses its abilities.

## Let's Begin!

<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/agents/04_autogen_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Install packages and set up Redis

In [9]:
%pip install -q autogen autogen_agentchat
%pip install -q "autogen-ext[redisvl]"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### For Colab download and run a Redis instance

In [10]:
# NBVAL_SKIP
'''
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes
'''

'\n%%sh\ncurl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg\necho "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list\nsudo apt-get update  > /dev/null 2>&1\nsudo apt-get install redis-stack-server  > /dev/null 2>&1\nredis-stack-server --daemonize yes\n'

In [11]:
import os
from typing import List
import requests
import json
from transformers import pipeline
import re
from collections import Counter

In [12]:
# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


## Building our agent

We'll be building a product review writing agent that takes in a set of product descriptions, collects relevant information, and provides a summary and analysis you can use for SEO.


### Defining tools
One of the defining the features of an agent is its ability to use tools so let's give it some.

For use with Autogen that just requires we define a well named function with type hints in its signature

In [13]:

async def summarize(restaurant_name: str, all_reviews: List[str]) -> str:
    """takes a list of reviews for a single restaurant and returns a summary of all of them."""
    # set up a summarizer model
    summarizer = pipeline('summarization', model='facebook/bart-large-cnn')
    # pass all the reviews
    summary = summarizer('\n'.join(all_reviews),  # concatenate all the reviews together
                max_length=1024,
                min_length=128,
                do_sample=False)[0]["summary_text"]
    return restaurant_name + ": " + summary


async def get_keywords(full_text: str) -> List[str]:
    """extract the most commonly occurring keywords present in the reviews to know
    which terms it is likely to rank highly for in keyword search engines."""
    # define a set of common English stopwords to ignore
    STOPWORDS = {
        'the', 'of', 'and', 'to', 'for', 'in', 'on', 'at', 'a', 'an', 'is', 'it', 'with', 'as', 'by', 'from', 'that',
        'this', 'be', 'are', 'was', 'were', 'or', 'but', 'not', 'so', 'if', 'then', 'than', 'which', 'who', 'whom',
        'about', 'into', 'out', 'up', 'down', 'over', 'under', 'again', 'further', 'once', 'here', 'there', 'when',
        'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no',
        'nor', 'only', 'own', 'same', 'can', 'will', 'just', 'don', 'should', 'now', 'has', 'have', 'had', 'do', 'does',
        'did', 'their', 'them', 'they', 'you', 'your', 'yours', 'he', 'him', 'his', 'she', 'her', 'hers', 'we', 'us',
        'our', 'ours', 'i', 'me', 'my', 'mine', 'also', 'place'
    }
    # remove punctuation and lowercase the text
    words = re.findall(r'\b\w+\b', full_text.lower())
    # filter out stopwords
    filtered_words = [word for word in words if word not in STOPWORDS]
    # count occurrences
    word_counts = Counter(filtered_words)
    # return the top 10
    return [word for word, _ in word_counts.most_common(10)]


## Adding relevant memories
Our agent needs to know what people think of these restaurants so we'll add the user reviews to our agent memory powered by Redis.


In [14]:
# fetch the reviews from our public S3 bucket
def fetch_data(file_name):
    dataset_path = 'datasets/'
    try:
        with open(dataset_path + file_name, 'r') as f:
            return json.load(f)
    except:
        url = 'https://redis-ai-resources.s3.us-east-2.amazonaws.com/recommenders/datasets/two-towers/'
        r = requests.get(url + file_name)
        if not os.path.exists(dataset_path):
            os.makedirs(dataset_path)
        with open(dataset_path + file_name, 'wb') as f:
            f.write(r.content)
        return json.loads(r.content.decode('utf-8'))

In [15]:
# the original dataset can be found here: https://www.kaggle.com/datasets/jkgatt/restaurant-data-with-100-trip-advisor-reviews-each

restaurant_data = fetch_data('factual_tripadvisor_restaurant_data_all_100_reviews.json')

print(f"we have {restaurant_data['restaurant_count']} restaurants in our dataset, with {restaurant_data['total_review_count']} total reviews")

restaurant_reviews = restaurant_data["restaurants"] # ignore the count fields

# drop some of the fields that don't need
for restaurant in restaurant_reviews:
    for field in ['region', 'country', 'tel', 'fax', 'email', 'website', 'address_extended', 'chain_name', 'trip_advisor_url']:
        if field in restaurant:
            restaurant.pop(field)

we have 147 restaurants in our dataset, with 14700 total reviews


In [16]:
from logging import WARNING, getLogger

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.memory.redis import RedisMemory, RedisMemoryConfig
from autogen_ext.models.openai import OpenAIChatCompletionClient

logger = getLogger()
logger.setLevel(WARNING)

# initailize Redis memory
redis_memory = RedisMemory(
    config=RedisMemoryConfig(
        redis_url="redis://localhost:6379",
        index_name="restaurant_reviews",
        prefix="trip_advisor",
    )
)

for restaurant in restaurant_reviews[:10]:
    # add each review to our agent memory
    for review in restaurant['reviews']:
        try:
            await redis_memory.add(
                MemoryContent(
                    content= " ".join([review.pop("review_title"), review.pop("review_text"), str({key: val for key, val in restaurant.items() if key != "reviews"})]),
                    mime_type=MemoryMimeType.TEXT,
                    metadata={},
                    #metadata={key: val for key, val in restaurant.items() if key != "reviews"},
                    #content=review.pop("review_title") +" " + review.pop("review_text"),
                    #mime_type=MemoryMimeType.TEXT,
                    #metadata={key: val for key, val in restaurant.items() if key != "reviews"},
                )
            )
        except KeyError:
            pass

## Create our agent and set it on a task

In [17]:

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Create assistant agent with ChromaDB memory
assistant_agent = AssistantAgent(
    name="restaurant_review_agent",
    system_message="you are an experienced writer and food critic. ",
    model_client=model_client,
    tools=[summarize , get_keywords],
    memory=[redis_memory],
    max_tool_iterations=5,
)

agent_task = "Write an article for me reviewing the restaurants in the San Francisco bay area. \
            Group them into categories based on their cuisine and price, and talk about the \
            top rated restaurants in each category. \
            After writing this, analyze your article and tell me the key search terms it is \
            likely to rank highly for. Call the function get_keywords()"


#assistant_agent.

stream = assistant_agent.run_stream(task=agent_task)
await Console(stream)


---------- TextMessage (user) ----------
Write an article for me reviewing the restaurants in the San Francisco bay area.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.             After writing this, analyze your article and tell me the key search terms it is             likely to rank highly for. Call the function get_keywords()
---------- MemoryQueryEvent (restaurant_review_agent) ----------
[MemoryContent(content="truly an experience Reserved a table based upon reviews I read.When my wife and I saw the menu, we had second thoughts it was unusual combinations and dishes.Since we were already there we ordered based upon the recommendation of our waitress.We were extremely happy we did.The food was excellent,If you are willing to try something a little off the usual. you will have a great experience. {'name': 'Bar Tartine', 'address': '561 Valencia St', 'locality': 'San Francisco', 'latitu

Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 488. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=244)
Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Device set to use mps:0
Your max_length is set to 1024, but your input_length is only 157. Since this is a summarization task, where outputs shorter than the input are typicall

---------- ToolCallExecutionEvent (restaurant_review_agent) ----------
[FunctionExecutionResult(content='Bar Tartine: The food was excellent,If you are willing to try something a little off the usual you will have a great experience. Wine list is extensive with wines from a broad range of countries. The food is delicious and unique. Each dish was filled with layers of flavors. I highly recommend the porridge bread, cheese dip, meatballs, sprouted lentil croquettes and the tomato salad. We really loved the spices and flavors, nothing bad on this menu. I would have given 5 stars but we got a hair in are food and we were still charged for the dish. The decor is really cool and the atmosphere is good. The service is excellent but this place is very expensive.', name='summarize', call_id='call_rPFZjRSlPUceLVO6WeL2wnaw', is_error=False), FunctionExecutionResult(content="Bazaar Cafe: The venue was okay, the food was okay and the value was okay. But none of it blew me away. Not really sure why

TaskResult(messages=[TextMessage(id='6bad5354-0a41-4a2b-a402-dc212d384e0b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 31, 20, 58, 26, 340453, tzinfo=datetime.timezone.utc), content='Write an article for me reviewing the restaurants in the San Francisco bay area.             Group them into categories based on their cuisine and price, and talk about the             top rated restaurants in each category.             After writing this, analyze your article and tell me the key search terms it is             likely to rank highly for. Call the function get_keywords()', type='TextMessage'), MemoryQueryEvent(id='546e0394-e4f9-4219-94bd-23692c08b29d', source='restaurant_review_agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 31, 20, 58, 26, 462659, tzinfo=datetime.timezone.utc), content=[MemoryContent(content="truly an experience Reserved a table based upon reviews I read.When my wife and I saw the menu, we had second th

In [21]:
stream =  assistant_agent.run_stream(task='now write a brief article discussing the unique points of each restaurant individually.')
await Console(stream)

---------- TextMessage (user) ----------
now write a brief article discussing the unique points of each restaurant individually.
---------- MemoryQueryEvent (restaurant_review_agent) ----------
[MemoryContent(content="Great for business or couples Creative food selection and fun bar area. We had a large group and the entire staff catered well. The food combinations were unique and well-presented. Overall, a good business dining experience in a gentrifying part of SF. {'name': 'Bar Tartine', 'address': '561 Valencia St', 'locality': 'San Francisco', 'latitude': 37.763972, 'longitude': -122.421755, 'cuisine': ['American', 'French', 'Contemporary', 'Bakery', 'Mediterranean'], 'price': '3', 'rating': 4.0, 'hours': {'tuesday': [['18:00', '22:00']], 'wednesday': [['18:00', '22:00']], 'thursday': [['18:00', '22:00']], 'friday': [['18:00', '23:00']], 'saturday': [['11:30', '14:30'], ['18:00', '23:00']], 'sunday': [['11:30', '14:30'], ['18:00', '22:00']]}, 'parking': True, 'parking_valet': True

TaskResult(messages=[TextMessage(id='bc759e29-0384-49b6-8405-be4477b46203', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 31, 21, 0, 25, 89718, tzinfo=datetime.timezone.utc), content='now write a brief article discussing the unique points of each restaurant individually.', type='TextMessage'), MemoryQueryEvent(id='30100736-3e67-467d-a2ac-5c2308922f96', source='restaurant_review_agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 31, 21, 0, 25, 279834, tzinfo=datetime.timezone.utc), content=[MemoryContent(content="Great for business or couples Creative food selection and fun bar area. We had a large group and the entire staff catered well. The food combinations were unique and well-presented. Overall, a good business dining experience in a gentrifying part of SF. {'name': 'Bar Tartine', 'address': '561 Valencia St', 'locality': 'San Francisco', 'latitude': 37.763972, 'longitude': -122.421755, 'cuisine': ['American', 'Frenc

### Clean up
close out our agent and empty our Redis index

In [ ]:
await model_client.close()
await redis_memory.close()